In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, TextClassificationPipeline
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator
import pandas as pd
from sklearn.metrics import classification_report

2024-04-14 12:50:55.201228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 12:50:55.201371: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 12:50:55.201532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 12:50:55.239712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 12:50:56.035552: W tensorflow/compiler/

In [2]:
echr = load_dataset("ecthr_cases",  "violation-prediction")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [5]:
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": "\n".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map(encode, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

In [6]:
train_df = pd.DataFrame(train_dataset)
train_df2 = Dataset.from_pandas(pd.concat([train_df[train_df['labels'] == 1].sample(762, random_state=42), train_df[train_df['labels'] == 0]]).sort_index())

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
training_args = TrainingArguments(
    output_dir="../models/distilbert_ecthr_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_df2,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.685452,0.697398
2,No log,0.572841,0.802168


TrainOutput(global_step=192, training_loss=0.5869803031285604, metrics={'train_runtime': 681.1464, 'train_samples_per_second': 4.475, 'train_steps_per_second': 0.282, 'total_flos': 403760631103488.0, 'train_loss': 0.5869803031285604, 'epoch': 2.0})

In [16]:
task_evaluator = evaluator("text-classification")
results_dict = {}
for metric in ["accuracy", "precision", "recall", "f1"]:
    results = task_evaluator.compute(
        model_or_pipeline="../models/distilbert_ecthr_model/checkpoint-192",
        data=test_dataset,
        metric=metric,
        tokenizer=tokenizer,
        strategy="simple",
        random_state=0,
        input_column='text',
        label_column='labels',
        label_mapping={"NON_VIOLATED": 0.0, "VIOLATED": 1.0},
    )
    metric_name, value = list(results.items())[0]
    results_dict[metric_name] = value

In [17]:
results_dict

{'accuracy': 0.717,
 'precision': 0.9504643962848297,
 'recall': 0.7098265895953757,
 'f1': 0.8127068166776968}

In [11]:
id2label = {0: "NON_VIOLATED", 1: "VIOLATED"}
label2id = {"NON_VIOLATED": 0, "VIOLATED": 1}
model = AutoModelForSequenceClassification.from_pretrained("../models/distilbert_ecthr_model/checkpoint-192", num_labels=2, id2label=id2label, label2id=label2id)
model.to(device)


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
inputs = tokenizer(test_dataset['text'], return_tensors="pt", truncation=True, padding=True).to(device)


In [13]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

RuntimeError: handle_0 INTERNAL ASSERT FAILED at "../c10/cuda/driver_api.cpp":15, please report a bug to PyTorch. 